In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from vocab import Vocabulary
from env import ConceptData
from create_data import addFile
import tensorflow as tf
import random
import numpy as np
import csv
import re
import os
import create_data
from xml.dom import minidom
import xml.etree.ElementTree as ET
from vocab import Vocabulary
from env2 import ConceptData
from create_data import addFile
import agents
from agents import find_lengths
import sender

In [2]:
sender_encoder = sender.Sender(num_options=10, batch_size=32)
sender_only_target = sender.SenderOnlyTarget(batch_size=32)
sender = sender.Sender_LSTM(agent = sender_encoder, vocab_size=99, embed_dim=44, num_cells=1, hidden_size=1, max_len=10, see_all_input=True)
receiver_decoder = agents.Receiver(num_options=10)
receiver_encoder = agents.Receiver_LSTM(agent=receiver_decoder, vocab_size=99, embed_dim=44, hidden_size=44, masking_value=99)

In [3]:
vocabulary = Vocabulary()

def addFile(name):
    file_name = name + "_structured_final.xml"
    file = minidom.parse(os.path.join(os.path.join('visa_dataset', 'UK'), file_name))
    concepts = file.getElementsByTagName('concept')

    for concept in concepts:
        vocabulary.addConcept(concept)


addFile("ANIMALS")
addFile("APPLIANCES")
addFile("ARTEFACTS")
addFile("CLOTHING")
addFile("CONTAINER")
addFile("DEVICE")
addFile("FOOD")
addFile("HOME")
addFile("INSTRUMENTS")
addFile("MATERIAL")
addFile("PLANTS")
addFile("STRUCTURES")
addFile("TOOLS")
addFile("TOYS")
addFile("VEHICLES")
addFile("WEAPONS")

for concept in vocabulary.concept_list:
    vocabulary.parseConcept(concept)
    
data = ConceptData(vocabulary, 9, 32)

In [4]:
input_concepts, sender_input, targets, receiver_input = data.getInput()
#print(sender_input.shape)

shape sender input:
(32, 10, 1, 595)


In [5]:
#sender_encoded = sender_encoder(sender_input)
message, log_prob_s, entropy_s, state_h = sender(sender_input)
print("message:")
print(message)
print("logits:")
print(log_prob_s)
print("entropy:")
print(entropy_s)
print("state_h:")
print(state_h)

32
input-shape:
(32, 10, 44)
message:
tf.Tensor(
[[48 74 47 34 83 67 64 64 19  0  0  0  0  0  0  0  0  0]
 [79 61 38 80 48 19 94 40 38  0  0  0  0  0  0  0  0  0]
 [71 30 87 59 41 37 61 43 20  0  0  0  0  0  0  0  0  0]
 [73 29 61 23 56 83 80 55 89  0  0  0  0  0  0  0  0  0]
 [23 96 59 12 74 23 96 90 85  0  0  0  0  0  0  0  0  0]
 [76 31 83 78 74 96 98 75 31  0  0  0  0  0  0  0  0  0]
 [61 31  5  5 22 47 16 62 36  0  0  0  0  0  0  0  0  0]
 [82 79 56 61 71 25  2  7 58  0  0  0  0  0  0  0  0  0]
 [12 35  4 70 68  4 69 91 88  0  0  0  0  0  0  0  0  0]
 [34 80 84 73 87 95  9 62 42  0  0  0  0  0  0  0  0  0]
 [62 22 55 44 92 37 55 51 15  0  0  0  0  0  0  0  0  0]
 [ 9  1 74 28 27  4 52 67 75  0  0  0  0  0  0  0  0  0]
 [58 93 74 11 39  1 78 85 49  0  0  0  0  0  0  0  0  0]
 [91 14 37 69 30 88 16 92 16  0  0  0  0  0  0  0  0  0]
 [57 71 35 21 97 88 94  0 89  0  0  0  0  0  0  0  0  0]
 [52 74 36 95 49 53  8 80 42  0  0  0  0  0  0  0  0  0]
 [38  5 22 96 35 13 71 66  3  0  0  0  

In [6]:

#einer- Dimension rausbekommen
receiver_input = tf.squeeze(tf.convert_to_tensor(receiver_input))
#richtige Reihenfolge der Dimensionen
receiver_input2 = tf.transpose(receiver_input, [1,0,2])
receiver_input2 = receiver_input2.numpy()

sample, log_prob_r, entropy_r, last_hidden = receiver_encoder(message=message, batch_size=32, max_len=10, receiver_input=receiver_input2)

print('entropy_r_shape ', entropy_r.shape)
print('entropy_s_shape ', entropy_s.shape)
message_lengths = find_lengths(message)
print('message_lengths: ', message_lengths)

effective_entropy_s = tf.zeros_like(entropy_r)
effective_entropy_s = np.array(effective_entropy_s)
effective_log_prob_s = tf.zeros_like(log_prob_r)
effective_log_prob_s = np.array(effective_log_prob_s)

#print(message.shape)

for k in range(message.shape[0]):
    for l in range(message.shape[1]):
        not_eosed = float(k < message_lengths[k])
        effective_entropy_s[k] += entropy_s[k, l] * not_eosed
        effective_log_prob_s[k] += log_prob_s[k, l] * not_eosed
        
effective_entropy_s = tf.convert_to_tensor(effective_entropy_s)
effective_log_prob_s = tf.convert_to_tensor(effective_log_prob_s)
        
receiver_output = []
for i in range(32):
    receiver_output.append(input_concepts[i][sample[i]])
    #print(targets[i] == receiver_output[i])

targets = tf.constant(targets)
receiver_output = tf.constant(receiver_output)

acc = np.sum(targets == receiver_output) / targets.shape[0]
acc

10
entropy_r_shape  (32,)
entropy_s_shape  (32, 18)
message_lengths:  [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 8, 10, 10, 10, 10, 10, 10, 10, 2, 10, 1, 10, 10, 10, 10, 10, 10, 10]


0.0625